### CLO IC Test (WIP)

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

pd.set_option('display.max_rows', None)

In [60]:
from absbox import Generic

bonds = (
    ("A1",{"balance":1000
             ,"rate":0.07
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Sequential":None}
             })
      ,("E",{"balance":500
             ,"rate":0.0
             ,"originBalance":500
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             })             
)

bond_OCs = [("A1",("A1",),1.1),("B",("A1","B"),1.05)]

OC_triggers = {f"OC_{tName}":
                  {"condition":(("/",("poolBalance",)
                                    ,("bondBalance",*b))
                                ,"<"
                                ,threshold)
                    ,"effects":None
                    ,"status":False
                    ,"curable":True}
                for (tName,b,threshold) in bond_OCs}

bond_ICs = [("A",("A1",),1.05),("B",("A1","B"),1.03)]

IC_triggers = {f"IC_{tName}":
                  {"condition":(("/",("accountBalance","intAcc")
                                    ,("bondDueInt",*b))
                                 ,"<"
                                 ,threshold)
                    ,"effects":None
                    ,"status":False
                    ,"curable":True}
                for (tName,b,threshold) in bond_OCs}

IC_triggerNames = list(IC_triggers.keys())
OC_triggerNames = list(OC_triggers.keys())




CLO_sample = Generic(
    "CLO_sample"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.02
          ,"remainTerm":30
          ,"status":"current"}]]}
    ,(("intAcc",{"balance":0}),("prinAcc",{"balance":0}))
    ,bonds
    ,(("seniorFee",{"type":{"annualPctFee":[("poolBalance",),0.003]}})
      ,("subFee",{"type":{"annualPctFee":[("poolBalance",),0.002]}})
      ,)
    ,{"amortizing":[
        # intereset waterfall
        # accrue and pay senior fee 
         ["calcAndPayFee","intAcc",['seniorFee']]
        # accrue interest for both bonds (NO payment) 
         ,["calcInt","A1","B"]
        # calculate OC and IC test
         ,['runTriggers',*IC_triggerNames]
        # payout interest of A1 
         ,["payInt","intAcc",["A1"]]
        # if all IC test is passing 
         ,["IfElse",[("trigger","InDistribution", "IC_A1"),False],
            # if passing
            [["payInt","intAcc",["B"]]
             ,["calcAndPayFee","intAcc",['subFee']]
             ,["payIntResidual","intAcc","E"]]
            # if failing, redemption senior to satisfy IC
            ,[["payPrin","intAcc",["A1"]
               ,{"limit":{"formula":("bondBalance","A1")}}
               ]
               # update IC test again
              ,['runTriggers',*IC_triggerNames]
               # if IC test is passing
              ,["If",[("trigger","InDistribution", "IC_A1"), False],
                # pay interest of B and residual to E
                ["payInt","intAcc",["B"]],
                ["calcAndPayFee","intAcc",['subFee']],
                ["payIntResidual","intAcc","E"]]
              ]]
        # principal waterfall
         ,["payPrinBySeq","prinAcc",["A1","B"]]
         ,["payPrinResidual","prinAcc",["E"]]
     ]}
    ,[["CollectedInterest","intAcc"]
      ,["CollectedPrincipal","prinAcc"]
      ,["CollectedPrepayment","prinAcc"]
      ,["CollectedRecoveries","prinAcc"]]
    ,None
    ,None
    ,None
    ,{"InDistribution":OC_triggers | IC_triggers}
    ,("PreClosing","Amortizing")
    )

In [64]:
from absbox import API,mkDeal,EnginePath
localAPI = API(EnginePath.DEV,check=False)

r = localAPI.run(CLO_sample
                ,poolAssump = None
                ,runAssump = None
                ,read=True)

Connecting engine server -> https://absbox.org/api/dev

✅Connected, local lib:0.28.7, server:0.28.11

Warning Message from server:
Bond E is not paid off

In [65]:
r['triggers']['InWF']['IC_A1']

,status,memo
date,,
2021-07-26,False,<Tag:1.74 L 1.10>
2021-08-20,True,<Tag:0.91 L 1.10>
2021-08-20,True,<Tag:0.00 L 1.10>
2021-09-20,True,<Tag:0.70 L 1.10>
2021-09-20,True,<Tag:0.00 L 1.10>
2021-10-20,True,<Tag:0.62 L 1.10>
2021-10-20,True,<Tag:0.00 L 1.10>
2021-11-20,True,<Tag:0.57 L 1.10>
2021-11-20,True,<Tag:0.00 L 1.10>


In [66]:
r['accounts']['intAcc'].loc["2021-07-26"]

,balance,change,memo
date,,,
2021-07-26,13.69,-0.23,<SeqPayFee:seniorFee>
2021-07-26,5.83,-7.86,<PayInt:A1>
2021-07-26,5.83,0.00,<PayInt:B>
2021-07-26,5.68,-0.15,<SeqPayFee:subFee>
2021-07-26,0.00,-5.68,<PayYield:E>


In [67]:
r['accounts']['intAcc'].loc["2021-08-20"]

,balance,change,memo
date,,,
2021-08-20,3.11,-0.07,<SeqPayFee:seniorFee>
2021-08-20,0.00,-3.11,<PayInt:A1>
2021-08-20,0.00,0.00,"[<PayPrin:A1>, <DS:CurrentBondBalanceOf [""A1""]>]"


In [68]:
r['bonds']['A1']

,balance,interest,principal,rate,cash,intDue,intOverInt,factor,memo
date,,,,,,,,,
2021-07-26,712.96,7.86,287.04,0.07,294.90,0.00,0,0.71296,"[<PayInt:A1>, <PayPrin:A1>]"
2021-08-20,640.90,3.11,72.06,0.07,75.17,0.30,0,0.64090,"[[<PayInt:A1>, <PayPrin:A1>], <PayPrin:A1>]"
2021-09-20,568.72,2.90,72.18,0.07,75.08,1.21,0,0.56872,"[[<PayInt:A1>, <PayPrin:A1>], <PayPrin:A1>]"
2021-10-20,496.42,2.80,72.30,0.07,75.10,1.68,0,0.49642,"[[<PayInt:A1>, <PayPrin:A1>], <PayPrin:A1>]"
2021-11-20,424.00,2.67,72.42,0.07,75.09,1.96,0,0.42400,"[[<PayInt:A1>, <PayPrin:A1>], <PayPrin:A1>]"
2021-12-20,351.46,2.57,72.54,0.07,75.11,1.82,0,0.35146,"[[<PayInt:A1>, <PayPrin:A1>], <PayPrin:A1>]"
2022-01-20,278.80,2.44,72.66,0.07,75.10,1.46,0,0.27880,"[[<PayInt:A1>, <PayPrin:A1>], <PayPrin:A1>]"
2022-02-20,206.02,2.33,72.78,0.07,75.11,0.78,0,0.20602,"[[<PayInt:A1>, <PayPrin:A1>], <PayPrin:A1>]"
2022-03-20,133.12,1.88,72.90,0.07,74.78,0.00,0,0.13312,"[<PayInt:A1>, <PayPrin:A1>]"
